In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import numpy as np
import numpy.linalg as la
from scipy.integrate import RK45

import sympy as sym

from table_maker import *
from functools import partial
from itertools import *
from math import ceil

import pickle

import helper_symbolics

from functools import partial

In [2]:
import matplotlib
matplotlib.rcParams.update({'font.size': 16})

Adapted from [Kilpatrick and Bressloff 2010](https://doi.org/10.1016/j.physd.2009.06.003)
$$\begin{align*}
    \mu u_t &= -u + \int_{-\infty}^\infty w(x,y) q(y,t) f\big( u(y,t)\big) \ dy \\
    \alpha q_t &= 1 - q - \alpha\beta q f(u) \\
\end{align*}$$

In [3]:
params = {
    'theta' :  0.2,
    'alpha' : 20.0,
    'beta'  :  0.25,
    'mu'    :  1.0
}

###############################################

a, b = -50, 100
n = 10**3 * 4

###############################################
xs = np.linspace(a,b,n)
h = (b-a)/(n-1)

def firing_rate(u, theta):
    return np.heaviside(u-theta, .5)
    
M = .5*np.exp( - np.abs( np.subtract.outer(xs, xs) ) ) * h

# def convolution(u):
#     return M@firing_rate(u)

def RK4_step(F, t, u, dt):
    k1 = F(t,u)
    k2 = F(t+dt/2, u + dt/2*k1)
    k3 = F(t+dt/2, u + dt/2*k2)
    k4 = F(t+dt, u + dt*k3)
    return u + dt/6*(k1+2*k2+2*k3+k4)

def euler_step(F, t, u, dt):
    return u + dt*F(t, u)

# u is a vector [u, q]
def F_with_params(t, u, theta, alpha, beta, mu):
    temp_var = firing_rate(u[0], theta)
    ret = np.array([
        1/mu * (-u[0] + M@(u[1]*temp_var)),
        (1-u[1])/alpha - beta*u[1]*temp_var
    ])
    return ret

In [7]:
F = partial(F_with_params, **params)
numerical_params = helper_symbolics.get_numerical_parameters(params, precision=15)
for k, v in numerical_params.items():
    print(f'{k} = {v}')
Unum, Qnum = helper_symbolics.get_traveling_pulse(numerical_params, validate=False)

\theta = 0.2000
\alpha = 20.00
\beta = 0.2500
\mu = 1.000
c = 1.03002856613383
\Delta = 9.34267426461180
A_0 = 0.9950189816633693
A_{-\Delta} = -0.002541507500080784
C1 = -0.09965323310971126


In [10]:
v1, v2 = helper_symbolics.get_adjoint_nullspace(numerical_params, validate=False)

In [11]:
u0 = np.zeros((2,n))

# def cos_bell(x, center=0, width=2*np.pi, height=1):
#     return (np.cos((x-center)/width*2*np.pi)+1)/2*height * np.heaviside(x-center+width/2,0) * np.heaviside(-x+center+width/2,0)
# u0[0] = cos_bell(xs, center=0, width=100)
# u0[1] += 1 # Set initial q to 1

u0[0] = Unum(xs-20)
u0[1] = Qnum(xs-20)

t0 = 0
t_final = 30
k = 1e-2

time_integrator = euler_step
# time_integrator = RK4_step

########################
steps = int(np.ceil( (t_final - t0)/k ))
k = (t_final - t0)/steps

us = [u0]
ts = [t0]

for step in range(steps):
    ts += [ts[-1] + k]
    us += [ time_integrator(F, ts[-1], us[-1], k) ]
    print('step %d/%d' % (step,steps), end='\r')

In [12]:
y_min = np.min(us)
y_max = np.max(us)
y_min -= .05*np.abs(y_max - y_min)
y_max += .05*np.abs(y_max - y_min)
window = y_min, y_max

stride = 20

fig, ax = plt.subplots(1, 1, figsize=(12,8))

line_u, = ax.plot(xs, us[0][0], 'b-', label="$u$")
line_q, = ax.plot(xs, us[0][1], 'g-', label="$q$")
ax.plot([a, b], [params['theta']]*2, 'k:')
ax.legend(loc='right')
ax.set_ylim(*window)
ax.set_xlim(0, b)
# ax.set_xlim(a,b)

def animate(i):
    print('step %d/%d' % (i,len(ts)), end='\r')
    line_u.set_ydata(us[i][0])
    line_q.set_ydata(us[i][1])
    return line_u,


# Init only required for blitting to give a clean slate.
def init():
    line_u.set_ydata(us[0])
    return line_u,

anim = animation.FuncAnimation(fig, animate, np.arange(0,len(ts),stride), init_func=init,
                              interval=1/24*1000, blit=True)

# anim.save('negative-feedback_pulse.mp4', writer='imagemagick', fps=24)

plt.close()
HTML(anim.to_html5_video())